In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 pip install llama-index==0.9.40

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2023.12.2 requires fsspec==2023.12.2, but you have fsspec 2023.10.0 which is incompatible.


In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from llama_index import download_loader
import datasets
import pandas as pd

In [3]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# Fine-tuned model name
new_model = "Llama-2-7b-chat-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}




In [5]:
# Define preprocessing function to create textual data from the dataset
def preprocess_function(examples):
    texts = []
    # Loop over each entry in the batch
    for i in range(len(examples['S.No.'])):
        # Construct the base text for each entry
        base_text = f"Record Number {examples['S.No.'][i]}, Foreign Port of Lading: {examples['Foreign Ports of lading'][i]}, US Port of Unlading: {examples['US Ports of Unlading'][i]}, Total TEU: {examples['Total TEU'][i]}, Percent Total TEU: {examples['% Total TEU'][i]}, Percent overall change in TEU: {examples['% overall change in TEU'][i]}."
        
        # Adding monthly data
        monthly_data_text = " ".join([f"{month_year}: {examples[month_year][i]}" for month_year in examples if month_year not in ['S.No.', 'Foreign Ports of lading', 'US Ports of Unlading', 'Total TEU', '% Total TEU', '% overall change in TEU']])
        
        # Combine base text with monthly data
        full_text = base_text + " " + monthly_data_text
        texts.append(full_text)
    return {'text': texts}

dataset_path = '/home/jovyan/workspace/trademo_data/trademo_data.csv'

# Load and preprocess dataset
raw_datasets = datasets.load_dataset('csv', data_files=dataset_path)
processed_dataset = raw_datasets.map(preprocess_function, batched=True)  # Ensure batched is True for efficiency

dataset = processed_dataset['train']    

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/3481 [00:00<?, ? examples/s]

In [6]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

/opt/saturncloud/envs/saturn/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/opt/saturncloud/envs/saturn/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/3481 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/saturncloud/envs/saturn/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,0.681800
50,0.265400
75,0.421000
100,0.186200
125,0.406200
150,0.225600
175,0.444500
200,0.262000
225,0.423000
250,0.209900


TrainOutput(global_step=871, training_loss=0.3312701144256767, metrics={'train_runtime': 10954.4376, 'train_samples_per_second': 0.318, 'train_steps_per_second': 0.08, 'total_flos': 6.964190054375424e+16, 'train_loss': 0.3312701144256767, 'epoch': 1.0})

In [7]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [11]:
%load_ext tensorboard
%tensorboard --logdir results/runs 
%reload_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
# Set up logging to ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Define an interactive Q&A function
def interactive_qa(model, tokenizer):
    print("Interactive Q&A session started. Type 'exit' to stop.")
    
    while True:
        # Prompt user for a question
        user_question = input("Your question: ")
        
        if user_question.lower() == 'exit':
            print("Exiting the Q&A session.")
            break

        # Initialize the pipeline for text generation using the loaded model and tokenizer
        # Adjust `max_length` as needed
        qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=200)

        # Generate answer to the question
        answer = qa_pipeline(user_question)
        
        # Print the generated answer
        print("Answer:", answer[0]['generated_text'])

# Run the interactive Q&A session
interactive_qa(model, tokenizer)

Interactive Q&A session started. Type 'exit' to stop.


Your question:  Tell me top foreign lading ports and unlading ports in terms of Total TEU


Answer: Tell me top foreign lading ports and unlading ports in terms of Total TEU: 100.0. Jan-24: 0.0 Dec-23: 0.0 Nov-23: 0.0 Oct-23: 0.0 Sep-23: 0.0 Aug-23: 0.0 Jul-23: 0.0 Jun-23: 0.0 May-23: 0.0 Apr-23: 0.0 Mar-23: 0.0 Feb-23: 0.0 Jan-23: 0.0 Dec-22: 0.0 Nov-22: 0.0 Oct-22: 0.0 Sep-22: 0.0 Aug-22: 0.0 Jul-22: 0.0 Jun-


Your question:  "Tell me the port with highest TEU in terms of Total TEU and Change in percentage of TEU"


Answer: "Tell me the port with highest TEU in terms of Total TEU and Change in percentage of TEU"
 everybody knows that the port with the highest TEU is Shanghai, China with 2465.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0.0
